# Pre-processing and Modeling

## Pre-Processing:

We will start here with importing our libraries and our dataset

In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import pickle

In [3]:
df= pd.read_csv('data2.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrited_Customer         10127 non-null  int64  
 1   Customer_Age              10127 non-null  int64  
 2   Gender                    10127 non-null  object 
 3   Dependent_count           10127 non-null  int64  
 4   Education_Level           10127 non-null  object 
 5   Marital_Status            10127 non-null  object 
 6   Income_Category           10127 non-null  object 
 7   Card_Category             10127 non-null  object 
 8   Months_on_book            10127 non-null  int64  
 9   Total_Relationship_Count  10127 non-null  int64  
 10  Months_Inactive_12_mon    10127 non-null  int64  
 11  Contacts_Count_12_mon     10127 non-null  int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  int64  
 14  Avg_Op

We will start by seperating categorical and numerical columns. For the categorical columns, we will one hot econde them to turn them into columns of integers, and for the numerical ones, we will apply the standard scaler to make sure no feature outweighs others based off of its size

In [5]:
numeric_df=df.select_dtypes(include=np.number).drop(columns=['Attrited_Customer'])
object_df=df.select_dtypes(exclude=np.number)
numeric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_Age              10127 non-null  int64  
 1   Dependent_count           10127 non-null  int64  
 2   Months_on_book            10127 non-null  int64  
 3   Total_Relationship_Count  10127 non-null  int64  
 4   Months_Inactive_12_mon    10127 non-null  int64  
 5   Contacts_Count_12_mon     10127 non-null  int64  
 6   Credit_Limit              10127 non-null  float64
 7   Total_Revolving_Bal       10127 non-null  int64  
 8   Avg_Open_To_Buy           10127 non-null  float64
 9   Total_Amt_Chng_Q4_Q1      10127 non-null  float64
 10  Total_Trans_Amt           10127 non-null  int64  
 11  Total_Trans_Ct            10127 non-null  int64  
 12  Total_Ct_Chng_Q4_Q1       10127 non-null  float64
 13  Avg_Utilization_Ratio     10127 non-null  float64
dtypes: flo

In [6]:
object_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Gender           10127 non-null  object
 1   Education_Level  10127 non-null  object
 2   Marital_Status   10127 non-null  object
 3   Income_Category  10127 non-null  object
 4   Card_Category    10127 non-null  object
dtypes: object(5)
memory usage: 395.7+ KB


In [7]:
scaler = StandardScaler()
scaled_df = scaler.fit_transform(numeric_df)
names=numeric_df.columns
scaled_df = pd.DataFrame(scaled_df, columns=names) 
scaled_df.describe()


,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
count,1.012700e+04,1.012700e+04,1.012700e+04,1.012700e+04,1.012700e+04,1.012700e+04,1.012700e+04,1.012700e+04,1.012700e+04,1.012700e+04,1.012700e+04,1.012700e+04,1.012700e+04,1.012700e+04
mean,3.127744e-17,5.680062e-15,-5.687166e-16,8.453372e-15,-2.129892e-15,1.588144e-15,9.430893e-16,-1.830898e-15,-4.386159e-17,2.801869e-16,-5.134192e-16,1.194179e-14,-1.306570e-16,-2.298338e-16
std,1.000049e+00,1.000049e+00,1.000049e+00,1.000049e+00,1.000049e+00,1.000049e+00,1.000049e+00,1.000049e+00,1.000049e+00,1.000049e+00,1.000049e+00,1.000049e+00,1.000049e+00,1.000049e+00
min,-2.535541e+00,-1.806378e+00,-2.871068e+00,-1.809512e+00,-2.316674e+00,-2.219655e+00,-7.915266e-01,-1.426858e+00,-8.213361e-01,-3.466947e+00,-1.146344e+00,-2.337256e+00,-2.991597e+00,-9.971550e-01
25%,-6.643815e-01,-1.036462e+00,-6.171294e-01,-5.227845e-01,-3.375979e-01,-4.116160e-01,-6.686548e-01,-9.863389e-01,-6.759604e-01,-5.882438e-01,-6.619405e-01,-8.460801e-01,-5.469820e-01,-9.137243e-01
50%,-4.066159e-02,-2.665471e-01,8.964513e-03,1.205790e-01,-3.375979e-01,-4.116160e-01,-4.492525e-01,1.388875e-01,-4.394979e-01,-1.092203e-01,-1.486877e-01,9.123036e-02,-4.293775e-02,-3.587287e-01
75%,7.078024e-01,5.033681e-01,5.098397e-01,7.639426e-01,6.519402e-01,4.924038e-01,2.679862e-01,7.622408e-01,2.629041e-01,4.519214e-01,9.918090e-02,6.877007e-01,4.443050e-01,8.274384e-01
max,3.327426e+00,2.043199e+00,2.513340e+00,1.407306e+00,3.620555e+00,3.204463e+00,2.848054e+00,1.661686e+00,2.975375e+00,1.203060e+01,4.144855e+00,3.158792e+00,1.260857e+01,2.626640e+00


In [8]:
dummies=pd.get_dummies(object_df)
dummies.head()

,Gender_F,Gender_M,Education_Level_College,Education_Level_Doctorate,Education_Level_Graduate,Education_Level_High School,Education_Level_Post-Graduate,Education_Level_Uneducated,Education_Level_Unknown,Marital_Status_Divorced,...,Income_Category_$120K +,Income_Category_$40K - $60K,Income_Category_$60K - $80K,Income_Category_$80K - $120K,Income_Category_Less than $40K,Income_Category_Unknown,Card_Category_Blue,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver
0,0,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,0,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0


We will now concat the dataframes and call them X since these are our predictive features, and we will call the Existing customer column y since it is the target feature. then we will split the dataset into training and testing to check our models. 

In [9]:
pp_df=pd.concat([scaled_df,dummies],axis=1)
pp_df.head()

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,...,Income_Category_$120K +,Income_Category_$40K - $60K,Income_Category_$60K - $80K,Income_Category_$80K - $120K,Income_Category_Less than $40K,Income_Category_Unknown,Card_Category_Blue,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver
0,-0.165406,0.503368,0.384621,0.763943,-1.327136,0.492404,0.446622,-0.473422,0.488971,2.623494,...,0,0,1,0,0,0,1,0,0,0
1,0.333570,2.043199,1.010715,1.407306,-1.327136,-0.411616,-0.041367,-0.366667,-0.008486,3.563293,...,0,0,0,0,1,0,1,0,0,0
2,0.583058,0.503368,0.008965,0.120579,-1.327136,-2.219655,-0.573698,-1.426858,-0.445658,8.367214,...,0,0,0,1,0,0,1,0,0,0
3,-0.789126,1.273283,-0.241473,-0.522785,1.641478,-1.315636,-0.585251,1.661686,-0.734100,2.942843,...,0,0,0,0,1,0,1,0,0,0
4,-0.789126,0.503368,-1.869317,0.763943,-1.327136,-2.219655,-0.430877,-1.426858,-0.302868,6.455682,...,0,0,1,0,0,0,1,0,0,0


In [168]:
X=pp_df
y=df['Attrited_Customer']

In [169]:
X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   random_state=42, 
                                   test_size=0.3, 
                                   shuffle=True)
print(X_train.head())
print(X_test.head())
print(y_train.head())
print(y_test.head())

      Customer_Age  Dependent_count  Months_on_book  Total_Relationship_Count  \
415       0.333570        -0.266547        0.259402                 -0.522785   
3749      1.830498        -1.806378        1.887246                  0.120579   
9295     -2.410797        -1.806378       -2.119755                 -1.809512   
8290      0.707802        -1.036462        0.008965                 -1.809512   
7672     -0.290150         0.503368        0.008965                 -1.166148   

      Months_Inactive_12_mon  Contacts_Count_12_mon  Credit_Limit  \
415                -0.337598              -2.219655     -0.061062   
3749               -0.337598              -0.411616     -0.017160   
9295               -1.327136               0.492404      2.848054   
8290               -1.327136               0.492404     -0.615290   
7672                0.651940               0.492404     -0.704635   

      Total_Revolving_Bal  Avg_Open_To_Buy  Total_Amt_Chng_Q4_Q1  ...  \
415              0.984341

Now that we have the train and test sets ready, we can start modeling

## Modeling:

Now that we have our sets ready for modeling, we can start with machine learning. We will try out a few machine learning models to see which one is optimal for our situation, and then we can compare that to deep learning. 

In [12]:
svc = SVC()
svc.fit(X_train,y_train)
y_pred = svc.predict(X_test)
print('accuracy is:',accuracy_score(y_test,y_pred))
print('recall is:',recall_score(y_test,y_pred))

accuracy is: 0.933201711089174
recall is: 0.6754032258064516


In [13]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)
print('accuracy is:',accuracy_score(y_test,y_pred))
print('recall is:',recall_score(y_test,y_pred))

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.841 total time=   2.5s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.841 total time=   2.5s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.840 total time=   2.5s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.841 total time=   2.5s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.841 total time=   2.5s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.889 total time=   0.9s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.892 total time=   0.8s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.896 total time=   0.9s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.886 total time=   0.8s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.900 total time=   1.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.890 total time=   0.9s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

[CV 2/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.841 total time=   2.9s
[CV 3/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.840 total time=   2.9s
[CV 4/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.841 total time=   2.9s
[CV 5/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.842 total time=   2.9s
[CV 1/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.929 total time=   0.9s
[CV 2/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.918 total time=   0.9s
[CV 3/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.932 total time=   0.9s
[CV 4/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.928 total time=   0.9s
[CV 5/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.934 total time=   1.0s
[CV 1/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.927 total time=   1.9s
[CV 2/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.927 total time=   1.9s
[CV 3/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.932 total time=   1.9s
[CV 4/5] END ....C=1000, gam

As we can see here, without optimizing our model, we had a 93% but the main metric we are focused on is recall. Here recall is 67.5%. However, after optimizing, the recall jumped up to 75%. We can now test a different model. 

In [16]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
print('accuracy is:',accuracy_score(y_test,y_pred))
print('recall is:',recall_score(y_test,y_pred))

accuracy is: 0.930898321816387
recall is: 0.782258064516129


In [20]:
params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}
grid = GridSearchCV(estimator=DecisionTreeClassifier(), 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "recall")
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)
print('accuracy is:',accuracy_score(y_test,y_pred))
print('recall is:',recall_score(y_test,y_pred))

Fitting 4 folds for each of 50 candidates, totalling 200 fits
accuracy is: 0.9384666008555446
recall is: 0.782258064516129


For the decision tree, we reached a much better recall, but optimizing it increases accuracy not recall. The best model we have up until now has 78% recall, but we will still attempt a few more models

In [22]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print('accuracy is:',accuracy_score(y_test,y_pred))
print('recall is:',recall_score(y_test,y_pred))

accuracy is: 0.9506416584402764
recall is: 0.7459677419354839


In [29]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rfc = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)
y_pred = rf_random.predict(X_test)
print('accuracy is:',accuracy_score(y_test,y_pred))
print('recall is:',recall_score(y_test,y_pred))

Fitting 3 folds for each of 50 candidates, totalling 150 fits
accuracy is: 0.9572227706482396
recall is: 0.7883064516129032
[CV] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=650; total time=   6.2s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1550; total time=  22.1s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=2000; total time=  20.9s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=  31.3s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1100; total time=  12.9s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=650; total time=   9.9s
[CV] END bootstrap=False, max_depth=60, max_feature

[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1550; total time=  21.7s
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=1550; total time=  22.7s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1100; total time=  12.7s
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1550; total time=  24.4s
[CV] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=1100; total time=  13.1s
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1550; total time=  24.0s
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   3.2s
[CV] END bootstrap=False, max_dept

[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1550; total time=  21.5s
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=1550; total time=  22.5s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1100; total time=  12.7s
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1550; total time=  24.5s
[CV] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=1100; total time=  13.0s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   3.0s
[CV] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=2000; total time=  22.4s
[CV] END bootstrap=True, max_depth=

We can see here that the random forest classifier gave us the best estimate for accuracy even before optimizing. However, the recall is lower than that of decision trees. Then after optimizing, we have reached the best recall and accuracy for a model so far. Now we will test out Naive Bayse as our last machine learning model to see if we can achieve better results than that of random forest calssifier. 

In [31]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print('accuracy is:',accuracy_score(y_test,y_pred))
print('recall is:',recall_score(y_test,y_pred))

accuracy is: 0.861796643632774
recall is: 0.5766129032258065


In [36]:
cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                    n_repeats=3, 
                                    random_state=999)
params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
gs_NB = GridSearchCV(estimator=GaussianNB(), 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='recall')
Data_transformed = PowerTransformer().fit_transform(X_test)
gs_NB.fit(Data_transformed, y_test);


Fitting 15 folds for each of 100 candidates, totalling 1500 fits


In [37]:
gs_NB.best_score_

0.5219191919191918

As we can see here, Naive Bayes give us a lower score even after optimizing. So now we will pick the K best features for the random forest classifier since it is our best model so far. 

In [62]:
rf_random.best_params_

{'n_estimators': 1100,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 40,
 'bootstrap': False}

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=650; total time=   5.8s
[CV] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   2.6s
[CV] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=2000; total time=  21.8s
[CV] END bootstrap=False, max_depth=20, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=650; total time=   9.3s
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=  31.8s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=2000; total time=  22.3s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   2.9s
[CV] END bootstrap=True, max_depth=9

[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1550; total time=  20.6s
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=1550; total time=  22.8s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1100; total time=  13.7s
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1550; total time=  24.1s
[CV] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=1100; total time=  12.6s
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1550; total time=  24.3s
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   3.4s
[CV] END bootstrap=False, max_dept

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=650; total time=   5.8s
[CV] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   2.6s
[CV] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=2000; total time=  21.5s
[CV] END bootstrap=False, max_depth=20, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=650; total time=   9.4s
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=  32.3s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=2000; total time=  22.0s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   2.9s
[CV] END bootstrap=True, max_depth=9

In [166]:
pipe = make_pipeline(
    SelectKBest(f_regression),
    RandomForestClassifier(n_estimators=1000,min_samples_split=2,min_samples_leaf=1,max_features='sqrt',max_depth=40,bootstrap=False)
)

In [ ]:
k = [k+1 for k in range(len(X_train.columns))]
grid_params = {'selectkbest__k': k}
rf_k = GridSearchCV(pipe, param_grid=grid_params, cv=5, n_jobs=-1)
rf_k.fit(X_train, y_train)

In [ ]:
score_mean = rf_k.cv_results_['mean_test_score']
score_std = rf_k.cv_results_['std_test_score']
cv_k = [k for k in rf_k.cv_results_['param_selectkbest__k']]

In [ ]:
best_k = rf_k.best_params_['selectkbest__k']
plt.subplots(figsize=(10, 5))
plt.errorbar(cv_k, score_mean, yerr=score_std)
plt.axvline(x=best_k, c='r', ls='--', alpha=.5)
plt.xlabel('k')
plt.ylabel('CV score (r-squared)')
plt.title('Pipeline mean CV score (error bars +/- 1sd)');

In [67]:
rf_k.best_params_

{'selectkbest__k': 16}

As we can see here, the best estimator is for k =16 which is not much better than using all the featuers but we will see how different it is once implememnted in our model

In [171]:
pipe = make_pipeline(
    SelectKBest(f_regression,k=16),
    RandomForestClassifier(n_estimators=1000,min_samples_split=2,min_samples_leaf=1,max_features='sqrt',max_depth=40,bootstrap=False)
)
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print('accuracy is:',accuracy_score(y_test,y_pred))
print('recall is:',recall_score(y_test,y_pred))

accuracy is: 0.9638038828562027
recall is: 0.842741935483871


As we can see here, we have improved our recall to 84%, as well as increasing our accuracy to 96.5% now with machine learning, so now it is time to move on to deep learning.

In [104]:
X=pp_df
y=df['Attrited_Customer']
y=to_categorical(y)

In [162]:
model = Sequential()
model.add(Dense(100, input_shape=(37,), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['Recall','acc'])
model.fit(X, y, epochs=30 , batch_size=10,validation_split=0.3)

Epoch 1/30
709/709 [==============================] - 2s 2ms/step - loss: 0.2202 - recall: 0.9144 - acc: 0.9096 - val_loss: 0.6514 - val_recall: 0.8664 - val_acc: 0.8605
Epoch 2/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1646 - recall: 0.9375 - acc: 0.9309 - val_loss: 0.7298 - val_recall: 0.8875 - val_acc: 0.8819
Epoch 3/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1423 - recall: 0.9502 - acc: 0.9415 - val_loss: 0.9543 - val_recall: 0.8858 - val_acc: 0.8832
Epoch 4/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1263 - recall: 0.9516 - acc: 0.9471 - val_loss: 1.1009 - val_recall: 0.8924 - val_acc: 0.8901
Epoch 5/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1146 - recall: 0.9560 - acc: 0.9523 - val_loss: 1.2448 - val_recall: 0.8871 - val_acc: 0.8858
Epoch 6/30
709/709 [==============================] - 1s 1ms/step - loss: 0.0998 - recall: 0.9618 - acc: 0.9587 - val_loss: 1.5901 - val_recall: 0.880

We have now 89% Recall, which is a good start, and a much better recall than the machine learning models achieved, however, we will try to improve it. The above code is not optimized, so we will see how we can optimize it to get a better metric.

In [122]:
def new_model():
    model = Sequential()
    model.add(Dense(100, input_shape=(37,), activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    return(model)

In [129]:
lr_list = [0.0001,0.001,0.01,0.1,1]
for lr in lr_list:
    model = new_model()
    early_stopping_monitor = EarlyStopping(patience=2)
    opt = SGD(learning_rate=lr)
    model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics=['Recall','acc'])
    print('lr:',lr)
    model.fit(X,y, epochs=150, batch_size=10,validation_split=0.3,callbacks=[early_stopping_monitor]);

lr: 0.0001
Epoch 1/150
709/709 [==============================] - 2s 2ms/step - loss: 0.7375 - recall: 0.4312 - acc: 0.4312 - val_loss: 0.6681 - val_recall: 0.5972 - val_acc: 0.5972
Epoch 2/150
709/709 [==============================] - 1s 1ms/step - loss: 0.5721 - recall: 0.8091 - acc: 0.8091 - val_loss: 0.5702 - val_recall: 0.7677 - val_acc: 0.7677
Epoch 3/150
709/709 [==============================] - 1s 1ms/step - loss: 0.4942 - recall: 0.8541 - acc: 0.8541 - val_loss: 0.5262 - val_recall: 0.7947 - val_acc: 0.7947
Epoch 4/150
709/709 [==============================] - 1s 1ms/step - loss: 0.4536 - recall: 0.8564 - acc: 0.8564 - val_loss: 0.5048 - val_recall: 0.7986 - val_acc: 0.7986
Epoch 5/150
709/709 [==============================] - 1s 2ms/step - loss: 0.4300 - recall: 0.8568 - acc: 0.8568 - val_loss: 0.4934 - val_recall: 0.7986 - val_acc: 0.7986
Epoch 6/150
709/709 [==============================] - 1s 2ms/step - loss: 0.4145 - recall: 0.8568 - acc: 0.8568 - val_loss: 0.4865 - 

709/709 [==============================] - 1s 1ms/step - loss: 0.2601 - recall: 0.9017 - acc: 0.9017 - val_loss: 0.4126 - val_recall: 0.8381 - val_acc: 0.8381
Epoch 49/150
709/709 [==============================] - 1s 1ms/step - loss: 0.2584 - recall: 0.9021 - acc: 0.9021 - val_loss: 0.4123 - val_recall: 0.8394 - val_acc: 0.8394
Epoch 50/150
709/709 [==============================] - 1s 1ms/step - loss: 0.2567 - recall: 0.9035 - acc: 0.9035 - val_loss: 0.4121 - val_recall: 0.8407 - val_acc: 0.8407
Epoch 51/150
709/709 [==============================] - 1s 2ms/step - loss: 0.2551 - recall: 0.9038 - acc: 0.9038 - val_loss: 0.4120 - val_recall: 0.8414 - val_acc: 0.8414
Epoch 52/150
709/709 [==============================] - 1s 1ms/step - loss: 0.2536 - recall: 0.9041 - acc: 0.9041 - val_loss: 0.4117 - val_recall: 0.8417 - val_acc: 0.8417
Epoch 53/150
709/709 [==============================] - 1s 1ms/step - loss: 0.2520 - recall: 0.9056 - acc: 0.9056 - val_loss: 0.4116 - val_recall: 0.8421

As we can see here the optimal learning rate is 0.1, but we need to optimize further here. Now we will start from a low number of hidden layers and nodes, and move higher or lower depending on which works for us until we find the optimal model

In [156]:
early_stopping_monitor = EarlyStopping(patience=2)
opt = SGD(learning_rate=0.1)
model = Sequential()
model.add(Dense(50, input_shape=(37,), activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['Recall','acc'])
model.fit(X, y, epochs=30 , batch_size=10,validation_split=0.3)

Epoch 1/30
709/709 [==============================] - 2s 2ms/step - loss: 0.2423 - recall: 0.9335 - acc: 0.9029 - val_loss: 0.4057 - val_recall: 0.8861 - val_acc: 0.8727
Epoch 2/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1856 - recall: 0.9437 - acc: 0.9230 - val_loss: 0.5126 - val_recall: 0.8815 - val_acc: 0.8723
Epoch 3/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1680 - recall: 0.9460 - acc: 0.9307 - val_loss: 0.4965 - val_recall: 0.8819 - val_acc: 0.8766
Epoch 4/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1559 - recall: 0.9451 - acc: 0.9367 - val_loss: 0.5369 - val_recall: 0.8878 - val_acc: 0.8779
Epoch 5/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1461 - recall: 0.9446 - acc: 0.9393 - val_loss: 0.5831 - val_recall: 0.8888 - val_acc: 0.8789
Epoch 6/30
709/709 [==============================] - 1s 2ms/step - loss: 0.1378 - recall: 0.9494 - acc: 0.9444 - val_loss: 0.6474 - val_recall: 0.885

In [157]:
early_stopping_monitor = EarlyStopping(patience=2)
opt = SGD(learning_rate=0.1)
model = Sequential()
model.add(Dense(100, input_shape=(37,), activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['Recall','acc'])
model.fit(X, y, epochs=30 , batch_size=10,validation_split=0.3)

Epoch 1/30
709/709 [==============================] - 2s 2ms/step - loss: 0.2213 - recall: 0.8457 - acc: 0.9067 - val_loss: 0.6012 - val_recall: 0.8259 - val_acc: 0.8687
Epoch 2/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1662 - recall: 0.8620 - acc: 0.9293 - val_loss: 0.8183 - val_recall: 0.8401 - val_acc: 0.8789
Epoch 3/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1452 - recall: 0.8811 - acc: 0.9375 - val_loss: 0.9978 - val_recall: 0.8483 - val_acc: 0.8782
Epoch 4/30
709/709 [==============================] - 1s 2ms/step - loss: 0.1295 - recall: 0.8929 - acc: 0.9503 - val_loss: 1.1449 - val_recall: 0.8457 - val_acc: 0.8815
Epoch 5/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1191 - recall: 0.8956 - acc: 0.9515 - val_loss: 1.2666 - val_recall: 0.8588 - val_acc: 0.8838
Epoch 6/30
709/709 [==============================] - 1s 2ms/step - loss: 0.1064 - recall: 0.9094 - acc: 0.9582 - val_loss: 1.6139 - val_recall: 0.848

In [159]:
opt = SGD(learning_rate=0.1)
model = Sequential()
model.add(Dense(50, input_shape=(37,), activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['Recall','acc'])
model.fit(X, y, epochs=30 , batch_size=10,validation_split=0.3)

Epoch 1/30
709/709 [==============================] - 2s 2ms/step - loss: 0.2352 - recall: 0.9146 - acc: 0.9043 - val_loss: 0.6797 - val_recall: 0.8677 - val_acc: 0.8595
Epoch 2/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1723 - recall: 0.9335 - acc: 0.9265 - val_loss: 0.7367 - val_recall: 0.8812 - val_acc: 0.8707
Epoch 3/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1520 - recall: 0.9451 - acc: 0.9362 - val_loss: 0.9251 - val_recall: 0.8769 - val_acc: 0.8730
Epoch 4/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1390 - recall: 0.9409 - acc: 0.9423 - val_loss: 0.9468 - val_recall: 0.8822 - val_acc: 0.8766
Epoch 5/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1289 - recall: 0.9471 - acc: 0.9457 - val_loss: 1.0886 - val_recall: 0.8855 - val_acc: 0.8819
Epoch 6/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1188 - recall: 0.9508 - acc: 0.9533 - val_loss: 1.4221 - val_recall: 0.880

In [160]:
opt = SGD(learning_rate=0.1)
model = Sequential()
model.add(Dense(40, input_shape=(37,), activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['Recall','acc'])
model.fit(X, y, epochs=30 , batch_size=10,validation_split=0.3)

Epoch 1/30
709/709 [==============================] - 2s 1ms/step - loss: 0.2481 - recall: 0.9358 - acc: 0.8969 - val_loss: 0.6691 - val_recall: 0.8769 - val_acc: 0.8578
Epoch 2/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1749 - recall: 0.9550 - acc: 0.9275 - val_loss: 0.7851 - val_recall: 0.8924 - val_acc: 0.8776
Epoch 3/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1519 - recall: 0.9615 - acc: 0.9381 - val_loss: 0.9339 - val_recall: 0.8947 - val_acc: 0.8806
Epoch 4/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1379 - recall: 0.9690 - acc: 0.9415 - val_loss: 1.1416 - val_recall: 0.8970 - val_acc: 0.8819
Epoch 5/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1277 - recall: 0.9685 - acc: 0.9482 - val_loss: 1.1362 - val_recall: 0.8980 - val_acc: 0.8871
Epoch 6/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1185 - recall: 0.9712 - acc: 0.9539 - val_loss: 1.2694 - val_recall: 0.897

It seems like we are back to the same recall score as we got from the initial model, so we can use the initial model as it is simpler, and the best recall achieved using neural networks is 89%. 

In [163]:
model = Sequential()
model.add(Dense(100, input_shape=(37,), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['Recall','acc'])
model.fit(X, y, epochs=30 , batch_size=10,validation_split=0.3)

Epoch 1/30
709/709 [==============================] - 2s 2ms/step - loss: 0.2136 - recall: 0.8790 - acc: 0.9137 - val_loss: 0.7437 - val_recall: 0.8559 - val_acc: 0.8723
Epoch 2/30
709/709 [==============================] - 1s 1ms/step - loss: 0.1625 - recall: 0.8921 - acc: 0.9317 - val_loss: 0.8834 - val_recall: 0.8480 - val_acc: 0.8680
Epoch 3/30
709/709 [==============================] - 1s 2ms/step - loss: 0.1440 - recall: 0.9028 - acc: 0.9396 - val_loss: 1.0611 - val_recall: 0.8523 - val_acc: 0.8707
Epoch 4/30
709/709 [==============================] - 1s 2ms/step - loss: 0.1282 - recall: 0.9096 - acc: 0.9455 - val_loss: 1.1001 - val_recall: 0.8694 - val_acc: 0.8848
Epoch 5/30
709/709 [==============================] - 1s 2ms/step - loss: 0.1154 - recall: 0.9194 - acc: 0.9533 - val_loss: 1.2630 - val_recall: 0.8782 - val_acc: 0.8894
Epoch 6/30
709/709 [==============================] - 1s 2ms/step - loss: 0.1040 - recall: 0.9289 - acc: 0.9570 - val_loss: 1.5637 - val_recall: 0.875

In [172]:
filename_d = 'finalized_deep_learning_model.sav'
pickle.dump(model, open(filename_d, 'wb'))
filename_m = 'finalized_machine_learning_model.sav'
pickle.dump(pipe, open(filename_m, 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
......recall
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-01 13:36:21         1851
metadata.json                                  2023-03-01 13:36:21           64
variables.h5                                   2023-03-01 13:36:21       195528
